## Importing and prepping data

In [1]:
import diff_classifier.aws as aws
import diff_classifier.pca as pca

In [ ]:
filename = 'features_test_01.csv'
folder = 'test_files'
aws.download_s3('{}/{}'.format(folder, filename), filename, bucket_name='nancelab.publicfiles')

In [ ]:
fstats = pd.read_csv(filename, encoding = "ISO-8859-1", index_col='Unnamed: 0')
nonnum = []
fstats_num = fstats.drop(nonnum, axis=1)
fstats_raw = fstats.as_matrix()
fstats

## PCA analysis

The pca.kmo function calculates the Kaiser-Meyer-Olkin statistic, a measure of sampling adequacy. Check the docstring for more information.

In [ ]:
kmostat = pca.kmo(fstats)

the pca.pca_analysis function provides a completely contained PCA analysis of the input trajectory features dataset. It includes options to impute NaN values (fill in with average values or drop them), and to scale features. Read the docstring for more information. 

In [ ]:
pcadataset = pca.pca_analysis(fstats, dropcols=nonnum)

## Visualization